# Práctica 1: Expresiones regulares y lexers

## 1. Realiza un sistema para identificar patrones dentro de cadenas textuales a partir de expresiones regulares

### a. Realiza una clase que simule a un autómata finito (puede ser no determinístico)

In [ ]:
class Automata(object):
    def __init__(self, states, initial, final, transitions):
        self.states = states
        self.initial = initial
        self.final = final
        self.transitions = transitions
    """Clase para objeto Autómata finito"""


    def accepts(self, cadena):
        q_actual = self.initial
        for caracter in cadena:
            if(q_actual, caracter) in self.transitions:
                q_actual = self.transitions[(q_actual, caracter)]
            else:
                return False
            return q_actual in self.final
    """Método que acepta o rechaza una cadena según si esta pertenece o no al lenguaje"""

    def match_token(self, texto):
        posicion_token = []
        for i in range(len(texto)):
            q_actual = self.initial
            for j in range(i, len(texto)):
                if (q_actual, texto[j]) in self.transitions:
                    q_actual = self.transitions[(q_actual, texto[j])]
                    if q_actual in self.final:
                        posicion_token.append((i, j+1))
                else:
                    break
        return posicion_token
    """Regresa la posición de un tóken cuando este es aceptado por el autómata"""

### b. Define la  función de construcción del autómata a partir de los patrones de expresiones regulares incluyendo los símbolos `| (or)`, `* (kleene)`, `+ (1 o más veces)` y `? (0 o 1 vez)`.

In [ ]:
def compile(regex: str) -> Automata:
    """Construye un autómata finito no determinista a partir de una expresión regular usando Thompson."""
    from collections import deque

    state_counter = 0
    def new_state():
        nonlocal state_counter
        state = state_counter
        state_counter += 1
        return state

    stack = []
    operators = []
    transitions = {}

    for char in regex:
        if char.isalnum():  # Caracteres normales
            s1, s2 = new_state(), new_state()
            transitions[(s1, char)] = {s2}
            stack.append((s1, {s2}))
        elif char == '|':  # Alternancia
            operators.append(char)
        elif char == '*':  # Cierre de Kleene
            s1, s2 = new_state(), new_state()
            old_s1, old_finals = stack.pop()
            transitions.setdefault((s1, ''), set()).update({old_s1, s2})
            for f in old_finals:
                transitions.setdefault((f, ''), set()).update({old_s1, s2})
            stack.append((s1, {s2}))
        elif char == '?':  # 0 o 1 ocurrencia
            old_s1, old_finals = stack.pop()
            s1, s2 = new_state(), new_state()
            transitions.setdefault((s1, ''), set()).update({old_s1, s2})
            for f in old_finals:
                transitions.setdefault((f, ''), set()).add(s2)
            stack.append((s1, {s2}))
        elif char == '+':  # 1 o más veces
            old_s1, old_finals = stack.pop()
            s1, s2 = new_state(), new_state()
            transitions.setdefault((s1, ''), set()).add(old_s1)
            for f in old_finals:
                transitions.setdefault((f, ''), set()).update({old_s1, s2})
            stack.append((s1, {s2}))

    # Procesar alternancia (|)
    while operators:
        op = operators.pop()
        if op == '|':
            s1, s2 = new_state(), new_state()
            right_s1, right_finals = stack.pop()
            left_s1, left_finals = stack.pop()
            transitions.setdefault((s1, ''), set()).update({left_s1, right_s1})
            for f in left_finals | right_finals:
                transitions.setdefault((f, ''), set()).add(s2)
            stack.append((s1, {s2}))

    initial, final_states = stack.pop()
    return Automata(set(range(state_counter)), initial, final_states, transitions)

### c. Prueba el autómata con el siguiente patrón y sobre las siguientes cadenas:

In [ ]:
from automata import compile, tokenize

# Construye el autómata de la regex
pattern = compile('niña|os?')

#Cadenas que acepta
print(pattern.accepts('niño'))
print(pattern.accepts('niña'))
print(pattern.accepts('niñas'))
print(pattern.accepts('niños'))
print(pattern.accepts('niñs'))

ModuleNotFoundError: No module named 'Automata'

### d. Utiliza el autómata para localizar los patrones que coinciden en el siguiente texto (la función tokenize utiliza los espacios en blanco para obtener tókens):

In [ ]:
#llamamos a las funciones
from automata import compile, tokenize

#definimos los lexers para tomar las palabras 1 niñas, 6 niño, 9 niña
pattern = compile('niña|os?')
# Tokenización del texto
text = tokenize('Las niñas extranjeras jugaban con el niño y la niña en el patio.')



# Encuentra las correspondecias
matches = pattern.match_token(text)

# Imprime índices y tokenes
# que cumplen el patrón dentro del texto
for i in matches:
    print(i, text[i])

NameError: name 'tokenize' is not defined

## 2. Utilizando lex, construye un lexer que pueda identificar los siguientes tókens

- Identificadores
- Operadores
    - binarios
    - unarios
    - relacionales
- Keywords
- Número
  - enteros
  - flotantes
- Literales
- Signos de puntuación
- Booleanos

In [ ]:
%option noyywrap
%{
#include <stdio.h>
%}

%%
[a-zA-Z_][a-zA-Z0-9_]*  { printf("IDENTIFICADOR: %s\n", yytext); }

\+|\-|\*|\/|%        { printf("OPERADOR_BINARIO: %s\n", yytext); }
\+\+|\-\-            { printf("OPERADOR_UNARIO: %s\n", yytext); }
==|!=|<=|>=|<|>       { printf("OPERADOR_RELACIONAL: %s\n", yytext); }

[ \t\n]              ; /* Ignorar espacios y saltos de línea */
.                   { printf("OTRO: %s\n", yytext); }
%%

int main() {
    printf("Ingresa una expresión:\n");
    yylex();
    return 0;
}
